# Grove LED Bar example

----
* [Introduction](#Introduction)
* [Setup the board](#Setup-the-board)
* [Setup LED Bar](#Setup-LED-Bar)
* [Use the LED Bar](#Use-the-LED-Bar)

----
## Introduction

This example shows how to use the Grove LED bar on the PYNQ-Z1 or PYNQ-Z2 board.

A [Grove LED Bar](http://www.seeedstudio.com/depot/Grove-LED-Bar-v20-p-2474.html) and Pynq Grove Adapter are required.

The LED bar is an example of a peripheral driven by GPIO.

The LED bar is attached to the G1 connection on the Pynq Grove Adapter, which is connected to PMODB on the board. 

The LED bar has 10 LEDs. 8 Green LEDs, 1 Orange LED, and 1 Red LED. The brightness for each LED can be set independently. All functions supporting the LED bar are implemented in the C code running on the IOP driving the LED bar. 

----
## Setup the board

In [ ]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

## Setup LED Bar
1. Connect the ***pmod2grove*** to ***PMODB***. 
2. Connect ***LED Bar*** to port ***G1*** of the Pynq Grove Adapter.

## Use the LED Bar

### 1. Instantiate and reset LED Bar
Create an instance of the LED bar 
The IOP (Pmod IOP (A or B) or Arduino IOP must be selected, along with the pins on the Pynq Grove adapter. 

In [ ]:
from pynq.lib.pmod import Grove_LEDbar
from pynq.lib.pmod import PMOD_GROVE_G1 # Import constants

# Instantiate Grove LED Bar on PMODB and on Pmod2Grove G1
# PMODA = 1, PMODB = 2, Arduino = 3
# Pynq Grove Adapter; G1 = [0,4], G2 = [1,5], G3 = [7,3], G4 = [6,2]
# The LED bar can be connected to G1-7 on the Pynq Arduino Shield:
# G1 = [2,3], G2 = [3,4], G3 = [4,5], G4 = [6,7], G5 = [8,9], G6 = [10,11], G7 = [12,13]
ledbar = Grove_LEDbar(base.PMODB,PMOD_GROVE_G1)
ledbar.reset()

### 2. Turn individual LEDs on or off

Write a 10-bit binary pattern, with each bit representing the corresponding LED. 1 = on, 0 = off

In [ ]:
#from pynq.pmods import pmod_const
from time import sleep

# Light up different bars in a loop
for i in range(2):
    ledbar.write_binary(0b1010100000)
    sleep(0.5)
    ledbar.write_binary(0b0000100100)
    sleep(0.5)
    ledbar.write_binary(0b1010101110)
    sleep(0.5)
    ledbar.write_binary(0b1111111110)
    sleep(0.5)

### 3. Set LEDs individually with different brightness levels

The brightness of each LED can be set individually by writing a list of 10x 8-bit values to the LED bar. 0 is off, 0xff is full brightness. 

In [ ]:
# Brightness 0-255
HIGH = 0xFF
MED  = 0xAA
LOW  = 0x01
OFF  = 0X00

brightness = [OFF, OFF, OFF, LOW, LOW, MED, MED, HIGH, HIGH, HIGH]

ledbar.write_brightness(0b1111111111,brightness)

### 4.  Set the "level" or the number of LEDs which are set 

A number or level of LEDs can be turned on, started from either end of the LED bar. For example, this could be used to indicate the level of something being measured.

write_level(level, bright_level, green_to_red)

* level is the number of LEDs that are on.
* bright_level [0-10] is the level of brightness
* green_to_red = 1 means the LEDs start being lit from the "green" end of the LED bar
* green_to_red = 0 means the LEDs start being lit from the "red" end of the LED bar.

e.g. ledbar.write_level(5,4,1) will light 5 LEDs, to brightness 4 (out of 10) and will start from the Green LED (the LED furthest away from Grove connector on the LED bar module.)


In [ ]:
for i in range (1,11):
    ledbar.write_level(i,3,0)
    sleep(0.3)
for i in range (1,10):
    ledbar.write_level(i,3,1)
    sleep(0.3)    

### 5. Simple example of controlling the LED Bar from the board buttons

The "level" of the LEDs can be set using buttons on the board. 

* Button 0 to increase level
* Button 1 to decrease level
* Button 3 to exit

In [ ]:
MAX_BTNS = 4

btns = [base.buttons[index] for index in range(4)] 
i = 0
ledbar.reset()

done = False
while not done:
    if btns[0].read() == 1:
        sleep(0.2)
        i = min(i+1,10)
        ledbar.write_level(i,2,1)
    elif btns[1].read() == 1:
        sleep(0.2)
        i = max(i-1,0)
        ledbar.write_level(i,2,1)
    elif btns[3].read() == 1:
        ledbar.reset()
        done = True